In [16]:
from tqdm import tqdm
import numpy as np
import time

a = np.array([1,2,3,4,5,6,7,8,9,10])
b = np.array([11,12,13,14,15,16,17,18,19,20])


process_bar = tqdm(enumerate(zip(a, b)),desc="My Test! -> ", total=len(a), ncols=100)
for idx_enum,data_enum in process_bar:
    process_bar.set_postfix("a=" a[idx_enum],b = b[idx_enum])
    process_bar.update()  # 默认参数n=1，每update一次，进度+n
    time.sleep(0.2)


SyntaxError: invalid syntax (985594614.py, line 11)

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import torch
import numpy as np

import sys
sys.path.append("D:\\OneDrive\\SUSTech\\MetaSurface_Lab\\Python Project\\D2NN_Simulation_COOPER\\Main_Model") 
print("sys_path = ",sys.path)
import Main_Model.model_define as modef

data_fashion = ['FASHION','fashion'] 
data_mnist = ['MNIST','mnist']
dataset_name = data_mnist


origin = np.load('dataset/'+dataset_name[0]+'/'+dataset_name[1]+'.npy', allow_pickle=True)
# [0][0] for train_data, [0][1] for train_label
# [1][0] for validation_data, [1][1] for validation_label
# [2][0] for test_data, [2][1] for test_label

origin_data = origin[1][0]
origin_label = origin[1][1]

label = np.load('dataset/'+dataset_name[0]+'_processed/label_validation_all.npy', allow_pickle=True)

data_dir = 'dataset/'+dataset_name[0]+'_processed/u0_validation'
data_file = 'u0_validation_'

idx = 6877
print("Label of ",idx," is ",label[:,idx])
print("Label of ",idx," is ",origin_label[idx])

# 测试所保存的模型
m_state_dict = torch.load('Main_Model/BD2NN_mnist_0_5_fashion_0_5_DEL_A_PROPA.pt')
D2NN = modef.OpticalNetwork(M=modef.M, L=modef.L, lmbda=modef.lmbda, z=modef.z).to(modef.device)
D2NN.load_state_dict(m_state_dict)
D2NN.eval()
print(D2NN)

plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(origin_data[idx,:,:])
plt.subplot(1, 2, 2)
plt.imshow(np.load(data_dir + "/" + data_file + "%d.npy"%idx, allow_pickle=True))
dataaa = torch.tensor(np.load(data_dir + "/" + data_file + "%d.npy"%idx, allow_pickle=True)).double().to(modef.device)
dataaa = dataaa.reshape(1, 250, 250)
with torch.no_grad():
    result = D2NN(dataaa, test=True, inverse=True)
print(result)

# plt.figure()
# plt.subplot(4, 2, 1)
# plt.imshow(np.load(data_dir + "/" + data_file + "%d.npy"%idx, allow_pickle=True))
# plt.subplot(4, 2, 2)
# plt.imshoworigin_data[idx,:,:]
# plt.subplot(4, 2, 3)
# plt.imshow(np.load(data_dir + "/" + data_file + "%d.npy"%(idx+1), allow_pickle=True))
# plt.subplot(4, 2, 4)
# plt.imshoworigin_data[idx+1,:,:]
# plt.subplot(4, 2, 5)
# plt.imshow(np.load(data_dir + "/" + data_file + "%d.npy"%(idx+2), allow_pickle=True))
# plt.subplot(4, 2, 6)
# plt.imshoworigin_data[idx+2,:,:]
# plt.subplot(4, 2, 7)
# plt.imshow(np.load(data_dir + "/" + data_file + "%d.npy"%(idx+3), allow_pickle=True))
# plt.subplot(4, 2, 8)
# plt.imshoworigin_data[idx+3,:,:]

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [5]:
# 不可迭代对象，手动更新
from tqdm import tqdm
import time

total = 200 #总迭代次数
loss = total

with tqdm(total=200, desc='Test') as pbar:
    pbar.set_description('Processing:') #  tqdm()中的desc参数和pbar.set_description()是一回事，后者优先。
    for i in range(20):
        loss -= 1
        pbar.set_postfix({'loss': '{0:1.5f}'.format(i/100), "x": '{0:1.5f}'.format(i)})  # 输入一个字典，显示实验指标
        # pbar.set_postfix(loss=i/100, x=i)
        pbar.update(10)  # 手动更新，默认参数n=1，每update一次，进度+n
        time.sleep(0.2)


Processing:: 100%|██████████| 200/200 [00:04<00:00, 48.90it/s, loss=0.19000, x=19.00000]
